<h1>Лабораторна робота 2</h1>
<h2>Гаврилюк Володимир ФБ-31</h2>

In [1]:
import urllib.request
import os
import datetime
import urllib
import pandas as pd
import glob

print("Set up!")

Set up!


<h2>Завдання 1</h2>

Для кожної із адміністративних одиниць України завантажити тестові
структуровані файли, що містять значення VHI-індексу. Ця процедура має бути
автоматизована, параметром процедури має бути індекс (номер) області. При
зберіганні файлу до його імені потрібно додати дату та час завантаження.

Передбачити повторні запуски скрипту, довантаження нових даних та колізію
даних;

In [2]:
def download_data(province_id, year_start=1981, year_end=2025):
    data_folder = "data"
    current_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
    if not os.path.exists(data_folder):
        os.makedirs(data_folder)
    else:
        existing = [file for file in os.listdir(data_folder) if f'vhi_id__{province_id}__{current_datetime[0:10]}' in file]
        if existing:
            print(f"This file already exists in {data_folder}")
            return

    url = f"https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1={year_start}&year2={year_end}&type=Mean"
    vhi_url = urllib.request.urlopen(url)

    filename = f'vhi_id__{province_id}__{current_datetime}.csv'
    file_path = os.path.join(data_folder, filename)

    with open(file_path, 'wb') as csvfile:
        csvfile.write(vhi_url.read())

    print(f"Downloaded {filename}")

In [3]:
    for i in range(1,28):
        download_data(i)

This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data
This file already exists in data


<h2>Завдання 2</h2>

Зчитати завантажені текстові файли у фрейм
(https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html)
Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не
повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути
реалізована у вигляді окремої процедури, яка на вхід приймає шлях до
директорії, в якій зберігаються файли;

In [4]:
def create_dataframe(folder):
    files = glob.glob(folder + '/*.csv')

    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    frames = []

    for file in files:
        region_id = int(file.split('__')[1])
        df = pd.read_csv(file, header=1, names=headers)
        df.at[0, "Year"] = df.at[0, "Year"][9:]
        df = df.drop(df.index[-1])
        df = df.drop(df.loc[df['VHI'] == -1].index)
        df = df.drop('empty', axis=1)
        df["area"] = region_id
        df['Week'] = df['Week'].astype(int)
        df['Year'] = df['Year'].astype(int)

        frames.append(df)

    result = pd.concat(frames).drop_duplicates().reset_index(drop=True)
    return result

In [5]:
data_frame = create_dataframe('data')
print(data_frame)

       Year  Week    SMN     SMT    VCI    TCI    VHI  area
0      1982     1  0.059  258.24  51.11  48.78  49.95    10
1      1982     2  0.063  261.53  55.89  38.20  47.04    10
2      1982     3  0.063  263.45  57.30  32.69  44.99    10
3      1982     4  0.061  265.10  53.96  28.62  41.29    10
4      1982     5  0.058  266.42  46.87  28.57  37.72    10
...     ...   ...    ...     ...    ...    ...    ...   ...
59422  2025    11  0.188  287.09  66.26   7.07  36.67     9
59423  2025    12  0.199  287.98  64.83  11.68  38.28     9
59424  2025    13  0.209  288.07  63.00  21.94  42.49     9
59425  2025    14  0.216  287.32  59.83  38.17  49.01     9
59426  2025    15  0.235  289.07  60.43  42.93  51.70     9

[59427 rows x 8 columns]


<h2>Завдання 3</h2>

Реалізувати окрему процедуру, яка змінить індекси областей, які використані на
порталі NOAA (за англійською абеткою) на наступні, за українською (виключно
старі індекси на нові):

In [6]:
eng_index_to_ukr_index = {
    1: 25, 2: 27, 3: 26,
    4: 1, 5: 4, 6: 5,
    7: 9, 8: 22, 9: 23,
    10: 24, 11: 11, 12: 10,
    13: 12, 14: 13, 15: 14,
    16: 15, 17: 16, 18: 17,
    19: 18, 20: 19, 21: 20,
    22: 21, 23: 7, 24: 2,
    25: 3, 26: 8, 27: 6
}

reg_id_name = {
    1: 'АР Крим', 2: 'Вінницька',  3: 'Волинська',  4: 'Дніпропетровська',  5: 'Донецька',  6: 'Житомирська',
    7: 'Закарпатська',  8: 'Запорізька',  9: 'Івано-Франківська', 10: 'Київ',  11: 'Київська',  12: 'Кіровоградська',
    13: 'Луганська',  14: 'Львівська',  15: 'Миколаївська',  16: 'Одеська',  17: 'Полтавська',
    18: 'Рівненська',  19: 'Севастополь', 20: 'Сумська',  21: 'Тернопільська',  22: 'Харківська',  23: 'Херсонська',
    24: 'Хмельницька',  25: 'Черкаська',  26: 'Чернівецька',  27: 'Чернігівська'
}

In [7]:
def replace_indexes(df):
    # Поточна дата
    current_datetime = datetime.datetime.now().strftime('%Y-%m-%d_%H-%M')
    save_path = f'cleaned_data_for_{current_datetime}.csv'

    if 'area' in df.columns:
        df['region_id'] = df['area'].map(eng_index_to_ukr_index)
        df.drop('area', axis=1, inplace=True)
    df['region_name'] = df['region_id'].map(reg_id_name)
    df['region_name'] = df['region_name'].astype(str)
    df = df.sort_values(by=['region_id', 'Year', 'Week'])
    df.reset_index(drop=True, inplace=True)

    existing_files = glob.glob(f'cleaned_data_for_{current_datetime[0:10]}_*.csv')
    if existing_files:
        print(f"The file for {current_datetime[0:10]} already exists. No new file saved.")
    else:
        df.to_csv(save_path, index=False, encoding='utf-8')
        print(f"Saved new file: {save_path}")

    return df

In [8]:
cleaned_data = replace_indexes(data_frame)
print(cleaned_data)

The file for 2025-04-20 already exists. No new file saved.
       Year  Week    SMN     SMT    VCI    TCI    VHI  region_id   region_name
0      1982     1  0.076  269.58  33.30  36.44  34.87          1       АР Крим
1      1982     2  0.070  269.26  30.50  37.03  33.76          1       АР Крим
2      1982     3  0.066  269.06  28.49  38.33  33.41          1       АР Крим
3      1982     4  0.063  269.28  25.78  39.94  32.86          1       АР Крим
4      1982     5  0.064  270.09  25.43  39.25  32.34          1       АР Крим
...     ...   ...    ...     ...    ...    ...    ...        ...           ...
59422  2025    11  0.173  281.15  71.95   5.78  38.86         27  Чернігівська
59423  2025    12  0.189  283.24  72.12   5.98  39.05         27  Чернігівська
59424  2025    13  0.201  284.81  70.73   7.45  39.09         27  Чернігівська
59425  2025    14  0.212  286.17  67.35  10.58  38.97         27  Чернігівська
59426  2025    15  0.221  286.67  60.22  21.16  40.69         27  Черніг

<h2>Завдання 4</h2>

Реалізувати процедури для формування вибірок наступного виду
(включаючи елементи аналізу):

- Ряд VHI для області за вказаний рік;

- Пошук екстремумів (min та max) для вказаних областей та років,
середнього, медіани;

- Ряд VHI за вказаний діапазон років для вказаних областей;

- Для всього набору даних виявити роки, протягом яких екстремальні
посухи торкнулися більше вказаного відсотка областей по Україні (20%
областей - 5 областей з 25). Повернути роки, назви областей з
екстремальними посухами та значення VHI;

In [9]:
def get_vhi_series_by_region_year(df, region_name, year):
    print(f"Getting VHI series for {region_name} in {year}....")
    return df[(df['region_name'] == region_name) & (df['Year'] == year)][['Week', 'VHI']]

In [10]:
def get_vhi_stats(df, region_names, years):
    filtered = df[(df['region_name'].isin(region_names)) & (df['Year'].isin(years))]
    stats = (
        filtered
        .groupby(['region_name', 'Year'])['VHI']
        .agg(['min', 'max', 'mean', 'median']) # groupby + aggregate - min max....
        .reset_index()
    )

    return stats

In [11]:
def get_vhi_series_by_region_year_range(df, region_names, start_year, end_year):
    return df[
        (df['region_name'].isin(region_names)) &
        (df['Year'] >= start_year) &
        (df['Year'] <= end_year)
    ][['Year', 'Week', 'region_name', 'VHI']]

In [12]:
def find_extreme_drought_years(df, threshold=15, min_regions=5):
    droughts = df[df['VHI'] < threshold]
    grouped = droughts.groupby(['Year', 'region_name']).size().reset_index(name='count')

    # для кожного року рахуємо кількість унікальних областей з посухою
    region_counts = grouped.groupby('Year')['region_name'].nunique()
    critical_years = region_counts[region_counts >= min_regions].index.tolist()

    # повертаємо повні дані для критичних років
    return droughts[droughts['Year'].isin(critical_years)][['Year', 'region_name', 'VHI']].sort_values('Year')


<h2>Виклики процедур</h2>

<h2>VHI Series</h2>

In [13]:
get_vhi_series_by_region_year(cleaned_data, "Київ", 2020)

Getting VHI series for Київ in 2020....


,Week,VHI
21735,1,38.33
21736,2,39.20
21737,3,40.19
21738,4,44.72
21739,5,47.36
21740,6,46.87
21741,7,45.34
21742,8,43.04
21743,9,40.56
21744,10,40.19


<h2>VHI Stats</h2>

In [14]:
get_vhi_stats(cleaned_data, ["Київська", "Харківська", "Київ"], [2015, 2016, 2017])

,region_name,Year,min,max,mean,median
0,Київ,2015,34.52,54.53,43.677308,42.430
1,Київ,2016,39.68,62.30,48.379615,47.815
2,Київ,2017,36.90,59.71,46.358077,46.010
3,Київська,2015,31.95,48.26,40.736154,41.055
4,Київська,2016,36.71,57.79,47.658654,46.920
5,Київська,2017,36.49,55.59,44.824423,43.810
6,Харківська,2015,26.55,60.22,45.839423,44.165
7,Харківська,2016,34.04,70.28,54.570769,54.495
8,Харківська,2017,29.44,58.87,45.783269,46.745


<h2>VHI Series by regions and years range</h2>

In [15]:
get_vhi_series_by_region_year_range(cleaned_data, ["Київська", "Харківська"], 2010, 2020)

,Year,Week,region_name,VHI
23416,2010,1,Київська,47.84
23417,2010,2,Київська,47.23
23418,2010,3,Київська,49.70
23419,2010,4,Київська,52.06
23420,2010,5,Київська,52.79
...,...,...,...,...
48194,2020,48,Харківська,36.12
48195,2020,49,Харківська,37.49
48196,2020,50,Харківська,36.90
48197,2020,51,Харківська,38.88


<h2>Droughts</h2>

In [16]:
find_extreme_drought_years(cleaned_data)

,Year,region_name,VHI
3151,2000,Вінницька,12.26
3152,2000,Вінницька,11.28
3154,2000,Вінницька,11.38
3153,2000,Вінницька,11.25
3156,2000,Вінницька,14.20
...,...,...,...
49699,2007,Херсонська,12.63
49698,2007,Херсонська,12.88
49697,2007,Херсонська,13.33
49703,2007,Херсонська,14.41
